### Documentation here https://dev.elsevier.com/documentation/ArticleRetrievalAPI.wadl
https://api.elsevier.com/content/article/doi/{doi}

In [1]:
import pandas as pd
import requests
import os
import xmltodict
import json
from _credentials import keys
import csv

In [ ]:
# Load the dataset
file_path = "csv_output\papers_included_537.csv"
df = pd.read_csv(file_path)

# Print the original number of rows for comparison
print(f"Original number of rows: {len(df)}")

# Remove duplicates based on the 'DOI' column
df_cleaned = df.drop_duplicates(subset=['DOI'])

# Print the number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_cleaned)}")

# Save the cleaned DataFrame to a new CSV file
cleaned_file_path = 'elsevier_search_results_cleaned.csv'
df_cleaned.to_csv(cleaned_file_path, index=False)

print(f"Cleaned data saved to {cleaned_file_path}")


df = pd.read_csv('elsevier_search_results_cleaned.csv')

# Creating a folder to save the downloaded articles
download_dir = 'papers_json'
os.makedirs(download_dir, exist_ok=True)

# Getting the total number of articles to download
total_articles = len(df)

# Initializing the counter for downloaded articles
downloaded_articles = 0

# Iterating through the list of DOIs
for index, row in df.iterrows():
    doi = row['DOI']  # Assuming there's a 'doi' column in the CSV file
    url = f"https://api.elsevier.com/content/article/doi/{doi}"
    
    # Sending the request
    response = requests.get(url, headers={
        "X-ELS-APIKey": keys["els-apikey"],

        "Accept": "application/json"
    })
    
    if response.status_code == 200:
        # Increment the counter if the download is successful
        downloaded_articles += 1
        article_data = response.json()
        filename = doi.replace('/', '_') + '.json'
        file_path = os.path.join(download_dir, filename)
        
        # Saving the article data to a file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(response.text)
    # Use '\r' to return to the start of the line and 'end=""' to prevent new line. Flush to ensure it's displayed immediately.
    print(f"\rDownloaded: {downloaded_articles}/{total_articles}", end='', flush=True)

# Adding a new line at the end of the process to ensure the command prompt appears correctly after the script finishes.
print("\nAll articles processed.")

Original number of rows: 554
Number of rows after removing duplicates: 549
Cleaned data saved to elsevier_search_results_cleaned.csv
Downloaded: 548/549
All articles processed.


In [2]:
import os
import json

directory = 'papers_json'

def process_json_files(directory):
    error_files = []

    # Iterate through each JSON file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)
                
                # Extract the original text from the JSON structure
                text = data['full-text-retrieval-response']['originalText']
                # Attempt to convert to lowercase to check for errors
                _ = text.lower()
            except AttributeError:
                error_files.append(filename)
                # Delete the file
                os.remove(file_path)
                print(f"Deleted {filename} due to an AttributeError.")
            except KeyError:
                print(f"KeyError: 'originalText' not found in {filename}")
    
    return error_files



# Run the processing function
error_files = process_json_files(directory)

# Print the problematic files
print("Deleted Error Files:")
for file in error_files:
    print(file)


Deleted Error Files:


# NLTK

IF it is the first time using this code, please download dependencies:
```python
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4') 
```

In [3]:
import os
import re
import json
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('punkt')
nltk.download('punkt_tab')


def preprocess_keyword(keyword):
    # Convert to lowercase
    keyword = keyword.lower()
    # Remove parentheses and their contents
    keyword = re.sub(r'\s*\([^)]*\)', '', keyword)
    # Tokenize words
    processed_words = nltk.word_tokenize(keyword)

    return processed_words

def extract_compound_keywords_from_json(folder_path):
    compound_keywords_dict = {}
    
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
                # Extract keywords from the JSON structure
                keywords_data = data.get('full-text-retrieval-response', {}).get('coredata', {}).get('dcterms:subject', [])
                keywords = [kw['$'] for kw in keywords_data if '$' in kw]
                
                for keyword in keywords:
                    processed_words = preprocess_keyword(keyword)
                    if len(processed_words) > 1:
                        compound_keyword = '_'.join(processed_words)
                        normal_keyword = ' '.join(processed_words)
                        compound_keywords_dict[normal_keyword] = compound_keyword
                    
    return compound_keywords_dict

def save_dict_as_py(dict_obj, output_file):
    # Convert dictionary to string and add import statement
    dict_content = f"compound_keywords = {dict_obj}\n"
    # Write to a .py file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(dict_content)
        
# Define the directory path that contains your JSON files
folder_path = 'json_papers'
compound_keywords = extract_compound_keywords_from_json(folder_path)

# Save dictionary to a .py file
output_file = '_cpwords.py'
save_dict_as_py(compound_keywords, output_file)
print(f"Dictionary has been saved to {output_file}")

Dictionary has been saved to _cpwords.py


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s2589602\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\s2589602\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
